In [32]:
import outside_helpers
import numpy as np
import outside_helpers
import psycopg2
import os
import json
import ast
from sklearn.cluster import KMeans
from django.utils import timezone
import math
current_path= os.getcwd()
with open(current_path + '/api_key_list.config') as key_file:
    api_key_list = json.load(key_file)
api_key = api_key_list["distance_api_key_list"]
conn_str = api_key_list["conn_str"]

In [82]:
def travel_outside_with_direction(origin_city, origin_state, target_direction, trip_len):
    poi_info = []
    conn = psycopg2.connect(conn_str)
    cur = conn.cursor()
    #coord_long, coord_lat
    cur.execute("SELECT index, coord_lat, coord_long FROM all_cities_coords_table WHERE city ='%s' AND state = '%s';" % (origin_city, origin_state))
    id_, start_lat, start_long = cur.fetchone()

    cur.execute("SELECT index, coord_lat, coord_long, adjusted_visit_length, ranking, review_score, num_reviews FROM poi_detail_table WHERE NOT (city = '%s' AND state = '%s') AND interesting = True AND ST_Distance_Sphere(geom, ST_MakePoint(%s,%s)) <= %s * 1609.34;" % (origin_city, origin_state, start_long, start_lat, trip_len))
    details = cur.fetchall()
    conn.close()
    for detail in details:
        if check_direction(start_lat, start_long, detail[1], detail[2], target_direction):
            poi_info.append(detail)
    return id_, start_lat, start_long, np.array(poi_info)


def check_direction(start_lat, start_long, outside_lat, outside_long, target_direction):
    angle_dict={"E": range(45, 135), "S": range(135, 215), "W": range(215, 305), "N": range(0, 45) + range(305, 360)}
    angle = calculate_initial_compass_bearing((start_lat, start_long), (outside_lat, outside_long))

    if int(angle) in angle_dict[target_direction]:
        return True
    else:
        return False

def calculate_initial_compass_bearing(pointA, pointB):
    """
    Calculates the bearing between two points.
    The formulae used is the following:
    theta = atan2(sin(delta(long)).cos(lat2),
                  cos(lat1).sin(lat2) − sin(lat1).cos(lat2).cos(delta(long)))
    :Parameters:
      - `pointA: The tuple representing the latitude/longitude for the
        first point. Latitude AND longitude must be in decimal degrees
      - `pointB: The tuple representing the latitude/longitude for the
        second point. Latitude AND longitude must be in decimal degrees
    :Returns:
      The bearing in degrees
    :Returns Type:
      float
    """
    if (type(pointA) != tuple) or (type(pointB) != tuple):
        raise TypeError("Only tuples are supported as arguments")

    lat1 = math.radians(pointA[0])
    lat2 = math.radians(pointB[0])

    diffLong = math.radians(pointB[1] - pointA[1])

    x = math.sin(diffLong) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1) * math.cos(lat2) * math.cos(diffLong))

    initial_bearing = math.atan2(x, y)

    # Now we have the initial bearing but math.atan2 return values
    # FROM -180° to + 180° which is not what we want for a compass bearing
    # The solution is to normalize the initial bearing as shown below
    initial_bearing = math.degrees(initial_bearing)
    compass_bearing = (initial_bearing + 360) % 360

    return compass_bearing

def create_outside_event_id_list(big_, medium_, small_):
    # print big_,medium_,small_
    event_type = ''
    if big_.shape[0] >= 1:
        if (medium_.shape[0] < 2) or (big_[0,3] >= medium_[0, 3]):
            if small_.shape[0] >= 6:
                event_ids = list(np.concatenate((big_[:1, 0], small_[0:6, 0]), axis=0))
            elif small_.shape[0] > 0:
                event_ids = list(np.concatenate((big_[:1, 0], small_[:, 0]), axis=0))
            else:
                event_ids = list(np.array(sorted(big_[0:, :], key=lambda x: (-x[1], x[2])))[:, 0])
            event_type = 'big'
        else:
            if small_.shape[0] >= 8:
                event_ids = list(np.concatenate((medium_[0:2, 0], small_[0:8,0]), axis=0))
            elif small_.shape[0] > 0:
                event_ids = list(np.concatenate((medium_[0:2, 0], small_[:,0]), axis=0))
            else:
                event_ids = list(np.array(sorted(medium_[0:, :], key=lambda x: (-x[1], x[2])))[:, 0])
            event_type = 'med'
    elif medium_.shape[0] >= 2:
        if small_.shape[0] >= 8:
            event_ids = list(np.concatenate((medium_[0:2, 0], small_[0:8, 0]), axis=0))
        elif small_.shape[0] > 0:
            event_ids = list(np.concatenate((medium_[0:2, 0], small_[:, 0]), axis=0))
        else:
            event_ids = list(np.array(sorted(medium_[0:, :], key=lambda x: (-x[1], x[2])))[:, 0])
        event_type = 'med'
    else:
        if small_.shape[0] >= 10:
            if medium_.shape[0] == 0:
                event_ids = list(np.array(sorted(small_[0:10, :], key=lambda x: (-x[1], x[2])))[:, 0])
            else:
                event_ids = list(np.array(sorted(np.vstack((medium_[:1, :], small_[0:10, :])), key=lambda x: (-x[1], x[2])))[:, 0])
        elif small_.shape[0] > 0:
            if medium_.shape[0] == 0:
                event_ids = list(np.array(sorted(small_[0:, :], key=lambda x: (-x[1], x[2])))[:, 0])
            else:
                event_ids = list(np.array(sorted(np.vstack((medium_, small_)), key=lambda x: (-x[1], x[2])))[:, 0])
        else:
            event_ids = list(np.array(sorted(medium_[0:, :], key=lambda x: (x[1], -x[2])))[:, 0])
        event_type = 'small'
    return event_ids, event_type

In [89]:
def assign_theme(details):
    theme_list_dict = {
    "family": ["Park", "Zoo", "Game"],
    "lifestyle": ["Nightlife", "Shopping", "Theater", "Food", "Spa", "Casino", "Show", "ShoppingMall"],
    "nature": ["StatePark", "NationalWildlifeRefuge", "NationalHistoricalPark", "NationalForest", "NationalMonument", "NationalMemorial"],
    "cultural": ["Landmark", "Museum", "OutdoorActivities", "Library", "Stadium"],
    "theme_park": ["ThemePark"],
    "national_park": ["NationalPark"],
    "other_list": ["Other", "VisotorCenter", "Transportation", "Tour", "Unuse_theater", "Unuse_transportation"]
    }

    assign_dict = {"family" : 0,"lifestyle": 0,"nature": 0,"cultural": 0,"theme_park": 0,"national_park": 0,"other_list": 0}

    assign_dict2 = {"family" : 0, "lifestyle": 0, "nature": 0, "cultural": 0, "theme_park": 0, "national_park": 0, "other_list": 0}

    assign_dict3 = {"family" : -1, "lifestyle": -1, "nature": -1, "cultural": -1, "theme_park": -1, "national_park": -1, "other_list": -1}

    assign_dict4 = {"family" : [], "lifestyle": [], "nature": [], "cultural": [], "theme_park": [], "national_park": [], "other_list": []}

    #create a list for each poi
    all_type = []
    for i in details:
        all_type.append([i["poi_type"], i["adjusted_visit_length"], i["num_reviews"], i["ranking"], i["review_score"]])

    for i in all_type:
        for key, value in theme_list_dict.iteritems():
            if i[0] in value: #locate the theme 
                assign_dict[key] += int(i[1]) #total time of theme
                assign_dict2[key] += int(i[2]) #total # of review of theme
                if assign_dict3[key] < 0:
                    assign_dict3[key] = int(i[3])
                else:
                    assign_dict3[key] = min(assign_dict3[key], int(i[3]))

                assign_dict4[key].append(float(i[4]))


    assign_dict = sort_dict(assign_dict) #order descending  by time

    # theme1 = assign_dict[0][1]
    # theme2 = assign_dict[1][1]
    # num_reviews = assign_dict2
    # ranking = assign_dict3
    if assign_dict[0][0] == assign_dict[1][0]: #check if the total time is same 
        if assign_dict2[assign_dict[0][1]] > assign_dict2[assign_dict[1][1]]:  #check number of review
            return [assign_dict[0][1], assign_dict2[assign_dict[0][1]], assign_dict3[assign_dict[0][1]], avg_list(assign_dict4[assign_dict[0][1]])]
        elif assign_dict2[assign_dict[0][1]] < assign_dict2[assign_dict[1][1]]:
            return [assign_dict[1][1], assign_dict2[assign_dict[1][1]], assign_dict3[assign_dict[1][1]], avg_list(assign_dict4[assign_dict[1][1]])]
        elif assign_dict3[assign_dict[0][1]] < assign_dict3[assign_dict[1][1]]: #check for ranking
            return [assign_dict[0][1], assign_dict2[assign_dict[0][1]], assign_dict3[assign_dict[0][1]], avg_list(assign_dict4[assign_dict[0][1]])]
        elif assign_dict3[assign_dict[0][1]] > assign_dict3[assign_dict[1][1]]:
            return [assign_dict[1][1], assign_dict2[assign_dict[1][1]], assign_dict3[assign_dict[1][1]], avg_list(assign_dict4[assign_dict[1][1]])]

    #return [theme, num of review, ranking, review_score]

    return [assign_dict[0][1], assign_dict2[assign_dict[0][1]], assign_dict3[assign_dict[0][1]], avg_list(assign_dict4[assign_dict[0][1]])]

def sort_dict(input_dict):
    temp_dict = [(input_dict[key], key) for key in input_dict]
    temp_dict.sort(reverse=True)
    return temp_dict

def avg_list(l):
    #for finding avg of review socre
    if len(l) != 0:
        return sum(l) / len(l)
    else:
        return 0

In [75]:
def outside_trip_poi(origin_city, origin_state, target_direction='N', n_days =1, full_day=True, regular=True, debug=True, username_id=1):
    outside_trip_id = '-'.join([str(origin_state.upper().replace(' ', '-')), str(origin_city.upper().replace(' ', '-')), target_direction,str(int(regular)), str(n_days)])
    origin_state = outside_helpers.check_state(origin_state)

    furthest_len = 100
    city_id, coord_lat, coord_long, city_infos = travel_outside_with_direction(origin_city, origin_state, 'N', 100)
#     if len(city_infos) <= 0:
#         conn = psycopg2.connect(conn_str)
#         cur = conn.cursor()
#         cur.execute('SELECT MAX(index) from outside_trip_table;')
#         new_index = cur.fetchone()[0] + 1
#         cur.execute("INSERT into outside_trip_table(index, username_id, outside_trip_id, outside_route_ids, event_id_lst, origin_city, origin_state, target_direction, n_routes, regular, full_day, outside_trip_details) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);", (new_index, username_id, outside_trip_id, '[]', '[]', origin_city, origin_state, target_direction, 0, regular, full_day, '[]'))
#         conn.commit()
#         conn.close()
#         print "finish update None for %s, %s, direction %s into database" % (origin_state, origin_city, target_direction)
#         return outside_trip_id, [], []
    poi_coords = city_infos[:, 1:3]
    return city_id, coord_lat, coord_long, city_infos

In [46]:
origin_city, origin_state= 'San Francisco', 'California'

In [76]:
city_id, coord_lat, coord_long, city_infos = outside_trip_poi(origin_city, origin_state)

In [64]:
import scipy.stats as scs
scs.describe(poi_details[:,6]).mean

142.28919860627178

In [80]:
poi_details[0]

array([ 927.       ,   38.5809738, -121.500259 ,  360.       ,
         26.       ,    4.       ,   49.       ])

In [68]:
n_routes = sum(1 for t in poi_details[:, 3] if t >= 120) / 10
n_routes = sum(1 for t in poi_details[:, 6] if t >= 142.3) / 5

In [69]:
kmeans = KMeans(n_clusters=n_routes).fit(poi_details[:,1:3])

In [71]:
route_labels = kmeans.labels_
outside_route_ids_list, outside_trip_details, details_theme, event_id_list =[], [], [], []
        

In [83]:
n_routes = 2
city_infos = poi_details
for i in range(n_routes):
    current_events, big_ix, med_ix, small_ix = [], [], [], []
    for ix, label in enumerate(route_labels):
        if label == i:
            time = city_infos[ix, 3]
            event_ix = city_infos[ix, 0]
            current_events.append(event_ix)
            if time > 180:
                big_ix.append(ix)
            elif time >= 120:
                med_ix.append(ix)
            else:
                small_ix.append(ix)

    big_ = outside_helpers.sorted_outside_events(city_infos, big_ix)
    med_ = outside_helpers.sorted_outside_events(city_infos, med_ix)
    small_ = outside_helpers.sorted_outside_events(city_infos, small_ix)
#     print 'big, med, sm', big_, med_
    event_ids, event_type = create_outside_event_id_list(big_, med_, small_)
    event_ids, event_type = outside_helpers.db_outside_event_cloest_distance(coord_lat, coord_long, event_ids=event_ids, event_type=event_type)
    event_ids, google_ids, name_list, driving_time_list, walking_time_list = outside_helpers.db_outside_google_driving_walking_time(city_id, coord_lat, coord_long, event_ids, event_type, origin_city=origin_city, origin_state=origin_state)
    event_ids, driving_time_list, walking_time_list, total_time_spent = outside_helpers.db_remove_outside_extra_events(event_ids, driving_time_list, walking_time_list)
#     outside_route_id = outside_trip_id + '-' + str(i)

#     if outside_helpers.check_outside_route_id(outside_route_id):
#         conn = psycopg2.connect(conn_str)
#         cur = conn.cursor()
#         cur.execute("DELETE FROM outside_route_table WHERE outside_route_id = '%s';" % (outside_route_id))
#         conn.commit()
#         conn.close()
    details = outside_helpers.db_outside_route_trip_details(event_ids, i)
    print i, details

0 [{'ranking': 14L, 'poi_type': 'Park', 'name': 'William Land Park', 'city': 'Sacramento', 'icon_url': 'https://s3.amazonaws.com/travel-with-friends/icon_img_file/icon_915.jpg', 'route': 0, 'img_url': 'https://s3.amazonaws.com/travel-with-friends/img_file/915.jpg', 'num_reviews': 125.0, 'state': 'California', 'coord_lat': 38.5361097, 'coord_long': -121.4985399, 'address': '1702 Sutterville Road, Sacramento, California, 95822, United States', 'check_full_address': 1L, 'adjusted_visit_length': 60.0, 'id': 915L, 'review_score': 4.5}, {'ranking': 9L, 'poi_type': 'Park', 'name': 'Sutter\xe2\x80\x99s Fort State Historic Park', 'city': 'Sacramento', 'icon_url': 'https://s3.amazonaws.com/travel-with-friends/icon_img_file/icon_910.jpg', 'route': 0, 'img_url': 'https://s3.amazonaws.com/travel-with-friends/img_file/910.jpg', 'num_reviews': 401.0, 'state': 'California', 'coord_lat': 38.5723062, 'coord_long': -121.4711749, 'address': '2701 L St, Sacramento, California, 95816, United States', 'check

In [92]:
for detail in details:
    print detail['poi_type']

StatePark
Food
Food
Landmark
Landmark


In [93]:
details

[{'address': '2400 London Ranch Rd, Glen Ellen, California, 95442, United States',
  'adjusted_visit_length': 60.0,
  'check_full_address': 1L,
  'city': 'Glen Ellen',
  'coord_lat': 38.3566642,
  'coord_long': -122.5409002,
  'icon_url': 'https://s3.amazonaws.com/travel-with-friends/icon_img_file/icon_14468.jpg',
  'id': 14468L,
  'img_url': 'https://s3.amazonaws.com/travel-with-friends/img_file/14468.jpg',
  'name': 'Jack London State Historic Park',
  'num_reviews': 345.0,
  'poi_type': 'StatePark',
  'ranking': 1L,
  'review_score': 4.5,
  'route': 1,
  'state': 'California'},
 {'address': '4026 Spring Mountain Rd, St. Helena, California, 94574, United States',
  'adjusted_visit_length': 60.0,
  'check_full_address': 1L,
  'city': 'St. Helena',
  'coord_lat': 38.5273618,
  'coord_long': -122.5535175,
  'icon_url': 'https://s3.amazonaws.com/travel-with-friends/icon_img_file/icon_7441.jpg',
  'id': 7441L,
  'img_url': 'https://s3.amazonaws.com/travel-with-friends/img_file/7441.jpg',
